# Khóa học LangGraph — Từ cơ bản đến nâng cao 🐍

Notebook này giúp bạn **tự học toàn bộ LangGraph** thông qua các phần:

1. Cài đặt & cấu hình môi trường  
2. Khái niệm State Graph & Message State  
3. Xây dựng graph cơ bản (linear)  
4. Thêm điều kiện rẽ nhánh & vòng lặp  
5. Tích hợp LangChain & OpenAI API  
6. Xây dựng Agent đa công cụ  
7. Quan sát & gỡ lỗi (visualize, callbacks)  
8. Bất đồng bộ & song song hoá  
9. Case Study: Pipeline RAG đơn giản  
10. Bài tập thực hành + gợi ý đáp án

> **Yêu cầu**: Python ≥ 3.10, API key OpenAI (hoặc provider tương đương).

## 1️⃣ Cài đặt gói cần thiết

In [ ]:
# Cài đặt LangGraph & phụ thuộc ( chỉ chạy lần đầu )
%pip install -q langgraph langchain openai tiktoken 

## 2️⃣ Khái niệm chính

* **StateGraph**: đồ thị xác định các nút (node) & cạnh (edge) cho luồng xử lý.  
* **MessageState**: đối tượng lưu trạng thái xuyên suốt khi graph chạy.  
* **Node**: hàm Python thuần (hoặc lớp) — xử lý State rồi trả State.  
* **Edge**: quy tắc điều hướng (mặc định tuần tự, có thể điều kiện).

## 3️⃣ Ví dụ: Graph tuyến tính tối giản

In [ ]:
from langgraph.graph import StateGraph, MessageState

# Bước 1 — Định nghĩa State class
class ChatState(MessageState):
    messages: list[str] = []

# Bước 2 — Khai báo các node
def greet(state: ChatState) -> ChatState:
    state.messages.append("Xin chào!")
    return state

def ask_name(state: ChatState) -> ChatState:
    state.messages.append("Bạn tên gì?")
    return state

# Bước 3 — Tạo đồ thị & thêm node, cạnh
graph = StateGraph(ChatState)
graph.add_node("greet", greet)
graph.add_node("ask_name", ask_name)
graph.set_entry_point("greet")
graph.add_edge("greet", "ask_name")

chat = graph.compile()

# Chạy thử
final_state = chat.invoke(ChatState())
print(final_state.messages)

## 4️⃣ Rẽ nhánh điều kiện

In [ ]:
from random import random

def random_branch(state: ChatState) -> str:
    return "joke" if random() < 0.5 else "fact"

def tell_joke(state: ChatState) -> ChatState:
    state.messages.append("🤡 Đây là một câu đùa!")
    return state

def fun_fact(state: ChatState) -> ChatState:
    state.messages.append("📚 Bạn biết không…")
    return state

graph2 = StateGraph(ChatState)
graph2.add_node("start", greet)
graph2.add_node("branch", random_branch)          # trả về id cạnh kế tiếp
graph2.add_conditional_edges("branch", {"joke": "joke", "fact": "fact"})
graph2.add_node("joke", tell_joke)
graph2.add_node("fact", fun_fact)
graph2.set_entry_point("start")

chat2 = graph2.compile()
print(chat2.invoke(ChatState()).messages)

## 5️⃣ Tích hợp LangChain & OpenAI

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

def llm_node(state: ChatState) -> ChatState:
    last_msg = state.messages[-1] if state.messages else "Xin chào"
    response = llm([HumanMessage(content=last_msg)]).content
    state.messages.append(response)
    return state

graph3 = StateGraph(ChatState)
graph3.add_node("user_prompt", ask_name)
graph3.add_node("llm_response", llm_node)
graph3.set_entry_point("user_prompt")
graph3.add_edge("user_prompt", "llm_response")

chat3 = graph3.compile()
print(chat3.invoke(ChatState()).messages[-1])

## 6️⃣ Quan sát & gỡ lỗi

In [ ]:
# Hiển thị cấu trúc graph dưới dạng DOT (có thể render bằng graphviz)
dot_code = graph3.get_graph().to_dot()
print(dot_code[:400], "...")  # in preview

## 7️⃣ Bất đồng bộ và song song hoá

In [ ]:
import asyncio

async def greet_async(state: ChatState) -> ChatState:
    await asyncio.sleep(0.2)
    state.messages.append("Hello (async)!")
    return state

g_async = StateGraph(ChatState)
g_async.add_node("start", greet_async, is_async=True)
g_async.set_entry_point("start")
chat_async = g_async.compile()

asyncio.run(chat_async.ainvoke(ChatState()))

## 8️⃣ Case Study: Pipeline RAG đơn giản

In [ ]:
# Giả lập bước truy vấn vector store → sinh câu trả lời
def retrieve(state: ChatState) -> ChatState:
    state.messages.append("🔍 Đã truy xuất 3 tài liệu liên quan.")
    return state

def answer(state: ChatState) -> ChatState:
    docs = " ".join(state.messages[-1:])
    reply = llm([HumanMessage(content=f"Trả lời dựa trên: {docs}")]).content
    state.messages.append(reply)
    return state

rag = StateGraph(ChatState)    .add_node("user", ask_name)    .add_node("retrieve", retrieve)    .add_node("answer", answer)    .set_entry_point("user")    .add_edge("user", "retrieve")    .add_edge("retrieve", "answer")    .compile()

print(rag.invoke(ChatState()).messages)

## 9️⃣ Bài tập

✏️ **Exercise 1**: Thêm node `translate` để dịch câu trả lời cuối sang tiếng Anh.  
✏️ **Exercise 2**: Thêm điều kiện: nếu tên người dùng chứa “AI” thì nhảy tới node `special_greet`.  
✏️ **Exercise 3**: Thử connect LangGraph với tool truy vấn cơ sở dữ liệu (SQL).

## 🔟 Tổng kết

* LangGraph cung cấp **stateful orchestration** mạnh mẽ cho agent.  
* Kết hợp LangChain + LangGraph giúp xây LLM workflow nâng cao.  
* Tiếp tục đọc docs chính thức & source code để nắm chi tiết.